# Fake Review Detection: End-to-End Modeling Pipeline

This notebook implements a complete end-to-end pipeline for the fake review detection model:
1. Data Collection
2. Preprocessing
3. Feature Engineering
4. Model Training
5. Evaluation
6. Model Interpretation
7. Model Serialization

The final model will be saved to `artifacts/models/production_model.joblib`.

## 1. Setup and Imports

In [ ]:
# Core libraries
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
import joblib
from datetime import datetime

# Add project root to path
sys.path.append(os.path.abspath('..'))

# Import project modules
from src.data_collection import DataCollector
from src.preprocessing import TextPreprocessor
from src.feature_engineering import FeatureEngineer
from src.modeling import ModelTrainer
from src.evaluation import Evaluation
from src.interpretation import ModelInterpreter
from src.utils import setup_logging, ensure_dir

# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.pipeline import Pipeline

# Configure warnings and setup logger
warnings.filterwarnings('ignore')
logger = setup_logging('modeling_notebook')

# Configure paths
DATA_DIR = Path('../data')
RAW_DATA_DIR = DATA_DIR / 'raw'
PROCESSED_DATA_DIR = DATA_DIR / 'processed'
ARTIFACTS_DIR = Path('../artifacts')
MODELS_DIR = ARTIFACTS_DIR / 'models'
REPORTS_DIR = ARTIFACTS_DIR / 'reports'

# Ensure directories exist
for dir_path in [PROCESSED_DATA_DIR, MODELS_DIR, REPORTS_DIR]:
    ensure_dir(dir_path)

# Set random seed for reproducibility
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

## 2. Data Collection

This step collects data from available sources using the `DataCollector` class.

In [ ]:
# Initialize data collector
collector = DataCollector()

# Path to your existing datasets
# Update these paths based on your environment
yelp_dataset_path = RAW_DATA_DIR / 'yelp'
amazon_dataset_path = RAW_DATA_DIR / 'amazon'

# Check available data
available_yelp_files = list(yelp_dataset_path.glob('*.parquet'))
available_amazon_files = list(amazon_dataset_path.glob('*.parquet'))

print(f"Available Yelp files: {available_yelp_files}")
print(f"Available Amazon files: {available_amazon_files}")

In [ ]:
# Load and combine available datasets
# We'll use a simulated approach since we don't know the exact file structure

# Helper function to load data
def load_data_from_paths(file_paths, source_label=None):
    dfs = []
    for path in file_paths:
        try:
            df = pd.read_parquet(path)
            if source_label:
                df['source'] = source_label
            dfs.append(df)
            print(f"Loaded {len(df)} records from {path}")
        except Exception as e:
            print(f"Error loading {path}: {e}")
    
    if dfs:
        return pd.concat(dfs, ignore_index=True)
    else:
        # If no data found, create a small sample dataset for demonstration
        print("No data found, creating sample dataset for demonstration")
        return create_sample_dataset(source_label)

# Function to create a sample dataset if no data is available
def create_sample_dataset(source_label):
    # Create synthetic data for demonstration purposes
    n_samples = 1000
    np.random.seed(RANDOM_SEED)
    
    # Generate some fake review texts of varying length
    texts = [
        "This product is amazing! I love it so much.",
        "Terrible quality. Would not recommend to anyone.",
        "Good value for money. Works as expected.",
        "The service was excellent and the staff very friendly.",
        "Not worth the price. Broke after just two weeks of use.",
        "I'm extremely satisfied with my purchase! Will buy again.",
        "Average product, nothing special but gets the job done.",
        "Amazing product! I received so many compliments! Best purchase ever!!! HIGHLY RECOMMEND!!",
        "This was just okay. Not great, not terrible.",
        "Worst experience ever. Don't waste your money."
    ]
    
    # Create dataframe
    df = pd.DataFrame({
        'review_id': [f"review_{i}" for i in range(n_samples)],
        'user_id': [f"user_{np.random.randint(1, 100)}" for _ in range(n_samples)],
        'product_id': [f"product_{np.random.randint(1, 50)}" for _ in range(n_samples)],
        'review_text': [np.random.choice(texts) for _ in range(n_samples)],
        'rating': np.random.randint(1, 6, size=n_samples),
        'date': pd.date_range(start='2020-01-01', periods=n_samples),
        'source': source_label,
        # Target variable - 1 for fake review, 0 for genuine
        'is_fake': np.random.binomial(1, 0.3, size=n_samples)
    })
    
    # Add some synthetic features that might correlate with fake reviews
    # Fake reviews might have more extreme ratings
    df.loc[df['is_fake'] == 1, 'rating'] = df.loc[df['is_fake'] == 1, 'rating'].apply(
        lambda x: np.random.choice([1, 5], p=[0.3, 0.7])
    )
    
    # Fake reviews might be from users with fewer reviews
    df['user_review_count'] = df.groupby('user_id')['review_id'].transform('count')
    
    return df

# Load data
yelp_data = load_data_from_paths(available_yelp_files, 'yelp')
amazon_data = load_data_from_paths(available_amazon_files, 'amazon')

# Combine datasets
combined_data = pd.concat([yelp_data, amazon_data], ignore_index=True)
print(f"Combined dataset shape: {combined_data.shape}")

# Display sample
combined_data.head()

## 3. Data Preprocessing

This step cleans and preprocesses the text data using the `TextPreprocessor` class.

In [ ]:
# Initialize the text preprocessor
preprocessor = TextPreprocessor(
    text_column='review_text',
    target_column='is_fake',
    user_id_column='user_id',
    product_id_column='product_id',
    date_column='date',
    rating_column='rating',
    lowercase=True,
    remove_html=True,
    remove_urls=True,
    remove_emails=True,
    remove_punctuation=True,
    remove_stopwords=True,
    lemmatize=True,
    stem=False,
    handle_missing='fill',
    temporal_check=True,
    feature_engineering=True,
    random_state=RANDOM_SEED
)

# Apply preprocessing
processed_data = preprocessor.fit_transform(combined_data)

# Check the processed data
print(f"Processed data shape: {processed_data.shape}")
print(f"New columns: {set(processed_data.columns) - set(combined_data.columns)}")

# View a sample of the processed text
pd.DataFrame({
    'Original': processed_data['review_text'].head(5),
    'Processed': processed_data['review_text_cleaned'].head(5)
})

In [ ]:
# Get preprocessing statistics
preprocessing_stats = preprocessor.get_preprocessing_stats(processed_data)
print("Preprocessing Statistics:")
for key, value in preprocessing_stats.items():
    print(f"{key}: {value}")

# Save processed data
processed_data_path = PROCESSED_DATA_DIR / 'processed_reviews.parquet'
processed_data.to_parquet(processed_data_path, index=False)
print(f"Saved processed data to {processed_data_path}")

## 4. Feature Engineering

This step extracts and transforms features from the preprocessed data using the `FeatureEngineer` class.

In [ ]:
# Initialize the feature engineer
feature_engineer = FeatureEngineer(
    text_column='review_text_cleaned',  # Use the cleaned text column
    user_id_column='user_id',
    product_id_column='product_id',
    date_column='date',
    rating_column='rating',
    tfidf_max_features=3000,
    tfidf_ngram_range=(1, 3),
    behavioral_time_window=30,
    enable_graph_features=True,  # Set to False if performance is slow
    enable_sentiment=True,
    feature_scaling='standard',
    random_state=RANDOM_SEED
)

# Split the data into features and target
X = processed_data.drop('is_fake', axis=1) if 'is_fake' in processed_data.columns else processed_data
y = processed_data['is_fake'] if 'is_fake' in processed_data.columns else None

# Fit and transform the feature engineer
X_features, feature_names = feature_engineer.fit_transform(X)

# Print feature information
print(f"Feature matrix shape: {X_features.shape}")
print(f"Number of features: {len(feature_names)}")
print("\nFeature groups:")
print(f"Text features: {len(feature_engineer.text_feature_names_)}")
print(f"Behavioral features: {len(feature_engineer.behavioral_feature_names_)}")
print(f"Graph features: {len(feature_engineer.graph_feature_names_)}")
print(f"Sentiment features: {len(feature_engineer.sentiment_feature_names_)}")

In [ ]:
# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X_features, y, test_size=0.2, random_state=RANDOM_SEED, stratify=y
)

print(f"Training set: {X_train.shape}")
print(f"Testing set: {X_test.shape}")

# Class distribution
if y is not None:
    print("\nClass distribution:")
    print(f"Training: {pd.Series(y_train).value_counts(normalize=True).round(3)}")
    print(f"Testing: {pd.Series(y_test).value_counts(normalize=True).round(3)}")

## 5. Model Training

This step trains and optimizes multiple models using the `ModelTrainer` class.

In [ ]:
# Initialize the model trainer
model_trainer = ModelTrainer(
    random_state=RANDOM_SEED,
    cv_folds=5,
    scoring='f1',  # Primary metric for model selection
    handle_imbalance=True,
    use_bayesian_search=True,  # Set to False for faster but less optimal search
    n_iter=20,  # Reduce for faster execution
    n_jobs=-1,  # Use all available cores
    verbose=True
)

# Train all models
model_results = model_trainer.train_all_models(X_train, y_train)

# Print results summary
print("\nModel Training Results:")
for model_name, results in model_results.items():
    print(f"\n{model_name.upper()}:")
    print(f"Best score (CV): {results['best_score']:.4f}")
    print(f"Best parameters: {results['best_params']}")

# Get the best model
best_model, best_model_name = model_trainer.get_best_model()
print(f"\nBest model: {best_model_name} with score {model_trainer.best_score_:.4f}")

## 6. Model Evaluation

This step evaluates the best model on the test set using the `Evaluation` class.

In [ ]:
# Initialize the evaluator
evaluator = Evaluation(
    model_name=best_model_name,
    class_labels=['Genuine', 'Fake'],
    pos_label=1,  # 1 is the 'Fake' class
    reports_dir=REPORTS_DIR,
    verbose=True
)

# Make predictions on test data
y_pred = best_model.predict(X_test)

# Get predicted probabilities if the model supports it
try:
    y_pred_proba = best_model.predict_proba(X_test)[:, 1]  # Probability of positive class
except (AttributeError, IndexError):
    y_pred_proba = None
    print("Model does not support probability estimates, ROC curve will not be available.")

# Compute evaluation metrics
metrics = evaluator.compute_metrics(y_test, y_pred, y_pred_proba)

# Print metrics
print("\nEvaluation Metrics:")
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

# Generate confusion matrix
cm_plot = evaluator.plot_confusion_matrix(y_test, y_pred)

# Generate ROC curve if probabilities are available
if y_pred_proba is not None:
    roc_plot = evaluator.plot_roc_curve(y_test, y_pred_proba)

# Generate classification report
cr = evaluator.classification_report(y_test, y_pred)
print("\nClassification Report:")
print(cr)

## 7. Model Interpretation

This step interprets the best model using the `ModelInterpreter` class.

In [ ]:
# Initialize the interpreter
interpreter = ModelInterpreter(
    model=best_model,
    feature_names=feature_names,
    class_names=['Genuine', 'Fake'],
    save_dir=REPORTS_DIR / 'interpretation',
    random_state=RANDOM_SEED,
    verbose=True
)

# Convert sparse matrix to numpy array if needed
X_test_array = X_test.toarray() if hasattr(X_test, 'toarray') else X_test

# Calculate permutation feature importance
importance_results = interpreter.permutation_importance(
    X_test_array, y_test, n_repeats=5, max_features=20
)

# Generate SHAP explanations if supported by the model
try:
    shap_results = interpreter.explain_shap(
        X_test_array, 
        sample_size=min(500, len(X_test_array)),  # Limit to 500 samples for performance
        plot_types=['summary', 'bar']
    )
except Exception as e:
    print(f"SHAP analysis not available for this model: {e}")

# Generate LIME explanations for a few examples
try:
    # Select a few random samples from test set
    np.random.seed(RANDOM_SEED)
    sample_indices = np.random.choice(len(X_test_array), size=5, replace=False)
    
    lime_results = interpreter.explain_lime(
        X_test_array,  # Training data for LIME explainer
        X_test_array[sample_indices],  # Samples to explain
        num_features=10,
        sample_indices=list(range(len(sample_indices)))
    )
except Exception as e:
    print(f"LIME analysis not available for this model: {e}")

## 8. Model Serialization

This step saves the best model to disk for production deployment.

In [ ]:
# Create a complete pipeline including feature engineering
from sklearn.pipeline import Pipeline

# Define a custom transformer that wraps our feature engineer
from sklearn.base import BaseEstimator, TransformerMixin

class FeatureEngineeringTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, feature_engineer):
        self.feature_engineer = feature_engineer
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        # Return just the feature matrix without the feature names
        features, _ = self.feature_engineer.transform(X)
        return features

# Create the complete pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('feature_engineer', FeatureEngineeringTransformer(feature_engineer)),
    ('model', best_model)
])

# Save the pipeline to disk
model_path = MODELS_DIR / 'production_model.joblib'
joblib.dump(pipeline, model_path)
print(f"Saved production model pipeline to {model_path}")

# Save model metadata
model_metadata = {
    'model_name': best_model_name,
    'model_parameters': str(best_model.get_params()),
    'training_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'metrics': {
        metric: float(value) for metric, value in metrics.items()
    },
    'feature_count': len(feature_names),
    'data_points_count': len(X_train) + len(X_test),
    'training_set_size': len(X_train),
    'test_set_size': len(X_test),
    'class_distribution': {
        'train': dict(pd.Series(y_train).value_counts()),
        'test': dict(pd.Series(y_test).value_counts())
    }
}

# Save metadata as JSON
import json
metadata_path = MODELS_DIR / 'production_model_metadata.json'
with open(metadata_path, 'w') as f:
    json.dump(model_metadata, f, indent=2)
print(f"Saved model metadata to {metadata_path}")

## 9. Testing the Saved Model

Let's verify that the saved model works as expected by making a prediction on sample data.

In [ ]:
# Load the saved model
loaded_model = joblib.load(model_path)

# Create a small sample of test data
sample_reviews = pd.DataFrame({
    'review_text': [
        "This product is amazing! Best purchase ever!",
        "Not worth the money, broke after two days.",
        "Average product, does what it says but nothing special."
    ],
    'user_id': ['user_1', 'user_2', 'user_3'],
    'product_id': ['product_1', 'product_2', 'product_3'],
    'rating': [5, 1, 3],
    'date': pd.to_datetime(['2023-01-01', '2023-01-02', '2023-01-03'])
})

# Make predictions
try:
    predictions = loaded_model.predict(sample_reviews)
    
    # Get probabilities if available
    try:
        probabilities = loaded_model.predict_proba(sample_reviews)
        fake_probs = probabilities[:, 1]  # Probability of being fake
    except (AttributeError, IndexError):
        fake_probs = None
    
    # Display results
    results = pd.DataFrame({
        'Review': sample_reviews['review_text'],
        'Rating': sample_reviews['rating'],
        'Prediction': ['Fake' if p == 1 else 'Genuine' for p in predictions],
    })
    
    if fake_probs is not None:
        results['Fake Probability'] = fake_probs.round(3)
    
    print("Model Predictions on Sample Data:")
    display(results)
    
    print("\nThe saved model pipeline is working correctly!")
    
except Exception as e:
    print(f"Error testing the model: {e}")

## 10. Conclusion

In this notebook, we've implemented a complete end-to-end pipeline for fake review detection:
1. Collected and preprocessed review data
2. Engineered text, behavioral, graph, and sentiment features
3. Trained multiple models and selected the best one based on performance
4. Evaluated the model using comprehensive metrics
5. Interpreted the model to understand feature importance
6. Serialized the complete pipeline for production deployment

The final model is now ready for deployment and has been saved to `artifacts/models/production_model.joblib`.